In [ ]:
from random import randint
from collections import Counter

In [ ]:
position_check_map = {
    "under_fire": "body",
    "harms_way": "fins",
    "neutral": "engines",
    "initiative": "guns",
}

plane_check_modfiers = {
    "engines": 1,
    "guns": 1,
    "fins": 0,
    "body": -1,
}

pilot_check_modfiers = {
    "engines": 0,
    "guns": 0,
    "fins": 0,
    "body": 0,
}

result_thresholds = {
    "fumble": 2,
    "fail": 5,
    "succeed": 10,
}


outcomes = {
    "under_fire": {
        "fumble": {
            "take_damage": 2,
            "heat": 1,
        },
        "fail": {
            "take_damage": 1,
            "move": "harms_way",
        },
        "succeed": {
            "move": "neutral"
        },
        "crit": {
            "move": "neutral",
            "modify": ("next_roll", 2),
        },
    },
    "harms_way": {
        "fumble": {
            "move": "harms_way",
            "heat": 1,
            "modify": ("next_roll", -2),
        },
        "fail": {
            "move": "under_fire",
            "heat": 1,
        },
        "succeed": {
            "move": "neutral",
        },
        "crit": {
            "move": "initiative",
        },
    },
    "neutral": {
        "fumble": {
            "move": "under_fire",
        },
        "fail": {
            "move": "harms_way",
        },
        "succeed": {
            "move": "initiative",
        },
        "crit": {
            "move": "initiative",
            "modify": ("next_roll", 2),
        },
    },
    "initiative": {
        "fumble": {
            "move": "harms_way",
            "heat": 0,
        },
        "fail": {
            "move": "neutral",
        },
        "succeed": {
            "hit": "damage"
        },
        "crit": {
            "hit": "damage",
            "hit": "damage",
        },
    },
}

starting_position = "neutral" # dynamically choose this via rolls

In [ ]:
class Plane:
    def __init__(self, check_modfiers, gun_damage= 3, hp=8):
        self.flight_check_modifiers = check_modfiers
        self.gun_damage = gun_damage
        self.hp = hp

class Pilot:
    def __init__(self, check_modfiers, hp=8):
        self.flight_check_modifiers = check_modfiers
        self.hp = hp

test_plane = Plane(plane_check_modfiers)
test_pilot = Pilot(pilot_check_modfiers)

In [ ]:
class PositionTracker:
    def __init__(self, starting_position, plane, pilot, outcomes):
        self.position = starting_position
        self.hp = plane.hp
        self.outcomes = outcomes
        self.flight_check_modifiers = self.combine_check_modifiers(plane.flight_check_modifiers, pilot.flight_check_modifiers)


        self.position_check_map = {
            "under_fire": "body",
            "harms_way": "fins",
            "neutral": "engines",
            "initiative": "guns",
        }
    
    def combine_check_modifiers(self, mod_a, mod_b):
        assert set(mod_a) == set(mod_b)

        check_modifiers_combined = {k: mod_a.get(k, 0) + mod_b.get(k, 0) for k in set(mod_a) & set(mod_b)}

        return check_modifiers_combined

    def get_stat_modifier(self):
        stat = self.position_check_map[self.position]
        modifier = self.flight_check_modifiers[stat]
        return modifier

    def get_heat_modifier(self):
        # not yet implemented
        modifier = 0
        return modifier

    def get_token_modifier(self):
        # not yet implemented
        modifier = 0
        return modifier

    def modify_roll(self, roll_value):
        stat_modifier = self.get_stat_modifier()
        heat_modifier = self.get_heat_modifier()
        token_modifier = self.get_token_modifier()

        return roll_value + stat_modifier + heat_modifier + token_modifier

    def determine_outcome(self, check_result):
        outcome = self.outcomes[self.position][check_result]
        return outcome

pt = PositionTracker(starting_position, test_plane, test_pilot, outcomes)


In [ ]:
def check_success(check_value, roll_value, result_thresholds, fumble_roll_val=1, crit_roll_val=12):
    """"""
    if roll_value == fumble_roll_val:
        result = "fumble"
    elif roll_value == crit_roll_val:
        result = "crit"
    elif check_value <= result_thresholds["fumble"]:
        result = "fumble"
    elif check_value <= result_thresholds["fail"]:
        result = "fail"
    elif check_value <= result_thresholds["succeed"]:
        result = "succeed"
    else:
        result = "crit"

    return result

In [ ]:
def roll(die=2, sides=6):
    "Roll die"
    result = 0
    for i in range(die):
        result += randint(1, sides)

    return result

In [ ]:
def check_roll(turn_tracker):
    """"""
    roll_value = roll()
    check_value = turn_tracker.modify_roll(roll_value)
    check_result = check_success(check_value, roll_value, result_thresholds)
    outcome = turn_tracker.determine_outcome(check_result)
    return outcome

In [ ]:
check_roll(pt)